In [3]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [5]:
import pandas as pd
df = pd.read_csv('asmr-20240924.csv', encoding='Windows-1252', sep=';', on_bad_lines='skip')
print(df.shape)
print(df.columns.tolist())

(9243, 9)
['$Code Evamed$', '$Motif de demande$', '$Code CIS$', '$Code CIP$', '$Dénomination spécialité$', '$Date avis définitif$', '$ASMR$', '$Valeur ASMR$', '$Libellé ASMR$ ']


In [7]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///asmr.db")
df.to_sql("asmr", engine, index=False, if_exists='replace')

9243

In [10]:
from sqlalchemy import create_engine, inspect

# Create an SQLite engine
engine = create_engine('sqlite:///asmr.db')

# Use the inspector to get a list of all tables in the database
inspector = inspect(engine)
tables = inspector.get_table_names()

# Print the list of tables
print("Existing tables:", tables)



Existing tables: ['asmr']


In [11]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM asmr LIMIT 5;"))

sqlite
['asmr']
[('$CT-21025$', '$Inscription (CT)$', '$66077218$', '$3400930294611$', '$FEXOFENADINE ARROW 180 mg, comprimé pelliculé$', '$04/09/2024$', '$ASMR$', '$V$', "$Ces spécialités sont des génériques qui n'apportent pas d'amélioration du service médical rendu (ASMR V) par rapport aux spécialités à base de fexofénadine déjà inscrites.$"), ('$CT-21016$', '$Inscription (CT)$', '$64314227$', '$3400955101734$', '$OXYGENE MEDICINAL LINDE HEALTHCARE 200 bar, gaz pour inhalation, en bouteille$', '$28/08/2024$', '$ASMR$', '$V$', "$Cette spécialité est un complément de gamme qui n'apporte pas d'amélioration du service médical rendu (ASMR V) par rapport aux présentations déjà inscrites.$"), ('$CT-21007$', '$Inscription (CT)$', '$60120260$', '$3400930280522$', '$CHLORHEXIDINE LIBERTY PHARMA 0,20 %, solution pour bain de bouche$', '$28/08/2024$', '$ASMR$', '$V$', "$Cette spécialité est un complément de gamme qui n'apporte pas d'amélioration du service médical rendu (ASMR V) par rapport aux

In [12]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

 ········


In [13]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [14]:
agent_executor.invoke({"input": "give me the first 5 rows of column named Valeur ASMR"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


asmr
Invoking: `sql_db_schema` with `{'table_names': 'asmr'}`



CREATE TABLE asmr (
	"$Code Evamed$" TEXT, 
	"$Motif de demande$" TEXT, 
	"$Code CIS$" TEXT, 
	"$Code CIP$" TEXT, 
	"$Dénomination spécialité$" TEXT, 
	"$Date avis définitif$" TEXT, 
	"$ASMR$" TEXT, 
	"$Valeur ASMR$" TEXT, 
	"$Libellé ASMR$ " TEXT
)

/*
3 rows from asmr table:
$Code Evamed$	$Motif de demande$	$Code CIS$	$Code CIP$	$Dénomination spécialité$	$Date avis définitif$	$ASMR$	$Valeur ASMR$	$Libellé ASMR$ 
$CT-21025$	$Inscription (CT)$	$66077218$	$3400930294611$	$FEXOFENADINE ARROW 180 mg, comprimé pelliculé$	$04/09/2024$	$ASMR$	$V$	$Ces spécialités sont des génériques qui n'apportent pas d'amélioration du service médical rendu (AS
$CT-21016$	$Inscription (CT)$	$64314227$	$3400955101734$	$OXYGENE MEDICINAL LINDE HEALTHCARE 200 bar, gaz pour inhalation, en bouteille$	$28/08/2024$	$ASMR$	$V$	$Cette spécialité est un complément d

{'input': 'give me the first 5 rows of column named Valeur ASMR',
 'output': 'The first 5 rows of the column named "Valeur ASMR" are all `$V$`.'}